# Question 4: General Theory/Application

_No need to be verbose, it's not fun for anyone_

1. What part of S**L**A**M** did this project deal with? Why? What does the other part deal with and how would it generally work, given that you only have LIDAR scans, RGB video stream, and noisy pose data for a moving robot?


2. Loop closures play an important role in reducing drift, how would you go about detecting these?


3. Explain the structure of your Jacobian. Is the pose-graph fully connected? Why/Why not?


4. With what you know now, how would you describe and differentiate the SLAM frontend and backend? Why do we need to optimise our poses/map in the first place - where does the noise come from/why?

_Your Answer_

## Answer 1

This project essentially deals with the **localisation** of SLAM. We only try to navigate in an unknown environment based on the odometry provided and the loop closure, control constraints. But there is not component that deals with the **mapping aspect** i.e. actually building the map of unknown environment.<br> 
Another aspect of comparison can be that we essentially only deal with the **backend** of SLAM and not the fornt-end. We are involved only in the processing of constrains(control, loop-closure), trajectory while not dealing with the front-ends aspects like feature adjustment, odometry, data association etc.<br>
Given that we have LiDAR scans, RGB video stream and noisy pose data; for the first comparison, building the map would involve state estimation, prediction and observation of landmarks using the sensor data. The mapping part will try to optimize the transforms of the point clouds given the landmark information obtained from the sensors? As the data is noisy, on multiple iterations this will be improved. 

## Answer 2
Loop closures are used to compensate for drift. Once the loop closures are properly found, it's used to backtrack drift errors caused in the SLAM implementation. It can be identified using NNs like PLce Net, Net VLAD can be used for detecting these. They are also performed by the global descriptor matching or Likelihood composition using SIFT etc. In a more intuitive sense, we can have two broad possiblities:geometric and feature based.<br>
    i)Geometry Based: It involved using the odometer to actually detect if a certain position has been passed before and identiying loop(not efficient and accruate enough due to errors).<br>
    ii)Feature Based: It involves similarity between two frames of images for detection which(error wont influence like the geometric case)

## Answer 3
The jacobian columns contains the partial derivative of the residual with respect to poses $x_i, y_i, theta_i$ where i spans over all vertices

$J = \begin{bmatrix}
    \frac{\partial f}{\partial x_{0}} \frac{\partial f}{\partial y_{0}} \frac{\partial f}{\partial \theta_{0}} \dots \frac{\partial f}{\partial x_{v}} \frac{\partial f}{\partial y_{v}} \frac{\partial f}{\partial \theta_{v}}\\
    \end{bmatrix}$
    
where 

$f = \begin{bmatrix}
    x_{0} + \Delta x_{(0,1)} \cos(\theta_0) - \Delta y_{(0,1)} \sin(\theta_0) - x_{1}\\
    y_{0} + \Delta y_{(0,1)} \cos(\theta_0) + \Delta x_{(0,1)} \sin(\theta_0) - y_{1}  \\
    \theta_{0}+  \Delta \theta_{(0,1)} -  \theta_{1} \\
    \vdots \\
    x_{p-1} + \Delta x_{(p-1,p)} \cos(\theta_{p-1}) - \Delta y_{(p-1,p)} \sin(\theta_{p-1}) - x_{p}\\
    y_{p-1} + \Delta y_{(p-1,p)} \cos(\theta_{p-1}) + \Delta x_{(p-1,p)} \sin(\theta_{p-1}) - y_{p}  \\
    \theta_{p-1}+  \Delta \theta_{(p-1,p)} -  \theta_{p} \\
x_{l[0]}+\Delta x_{(l[0],l[1])}\cos(\theta_{l[0]})-\Delta y_{(l[0],l[1])}\sin(\theta_{l[0]})-x_{l[1]}\\
y_{l[0]}+\Delta y_{(l[0],l[1])}\cos(\theta_{l[0]})+\Delta x_{(l[0],l[1])}\sin(\theta_{l[0]})-y_{l[1]}\\
    \theta_{l[0]}+  \Delta \theta_{(l[0],l[1])} -  \theta_{l[1]} \\
    \vdots \\
x_{l[q-1]}+\Delta x_{(l[q-1],l[q])}\cos(\theta_{l[q-1]})-\Delta y_{(l[q-1],l[q])} \sin(\theta_{l[q-1]}) - x_{l[q]}\\
    y_{l[q-1]} + \Delta y_{(l[q-1],l[q])} \cos(\theta_{l[q-1]}) + \Delta x_{(l[q-1],l[q])} \sin(\theta_{l[q-1]}) - y_{l[q]}  \\
    \theta_{l[q-1]}+  \Delta \theta_{(l[q-1],l[q])} -  \theta_{l[q]} \\
    x_{0}  - a\\
    y_{0} - b  \\
    \theta_{0} -  c \\
    \end{bmatrix}$

The shape of the jacobian is (3p + 3q + 3) X v v is number of vertices, p is number of odometry constraints, q is number of loop constraints.

The pose graph **is not fully connected**. This is because odometry edges only connect adjacent vertices and there are only a few (20 to be exact) loop closure edges that connect certain vertices. There exist many vertices that are not connected to each other (for example vertex 0 and vertex 3). This is the case with most SLAM problems(sparsity of SLAM).

For a fully connected graph there would have ** $n*(n-1)/2$ ** constrants/edges where n is the number of vertices. We see that there aren't that many constraints in our given problem

## Answer 4

**SLAM front end deals with feature extraction and data association.** It takes the sensor data and gives a set of features for our robot (eg. x,y,theta). Also through sensor information, it gives us odometry and loop closure edges, which is data association between each pose of the robot.  

**SLAM back end deals with localization and mapping**. It takes the poses and associations given by the front end and optimizes localization and mapping for our robot.

We need to optimise poses/map because the *different data associations given by the SLAM frontend are not always correct. The odometry edges and the loop closure edges have **different susceptibility to Noise**. This leads to conflicting information. If we construct our poses only following the odometry constraints, then the loop closure constraints are often broken. Hence we need to get a balance between following both these constraints and the anchor (fixing the coordinate frame) given. This is why optimisation is important.

Noise comes from various sources. One possibility is through sensor readings which are not always accurate. We know that when we are finding the pose of the robot through successive transformations, when it is moving, the error accumulates and the poses are no longer correct. There might also be slight error in loop closure.